# Optimization: Operation Room Scheduling

### Goal:
A hospital is relocating to a larger hospital with five operating rooms. Relocation will increase weekly operating hours from 190 to 213.5 hours. The task is to create a new schedule for six departments that allocate slots in an equitable way. 'Equitable' here means each department maintaining the current share (%) of operating hours and minimize total underallocation (%) of each department to operating rooms.

#### (1) Optimization Model

$$
\begin{align}
s_i &= \text{underallocation of department } i \text{ (total number of hours below the target)} \\
t_i &= \text{target hours that department } i \text{ should receive} \\
x_i &= \text{the new no. of hours allocated to department } i \\
y_{ijk} &= 1 \text{ if department } i \text{ is allocated to room } j \text{ in day } k, 0 \text{ otherwise} \\
\end{align}
$$


$$
\begin{align}
\underset{{\bf s}}{\text{min}} \;\; & \sum_{i = 1}^6 \dfrac{s_i}{t_i} \\
\text{s.t.} \;\; & s_i =\begin{cases} 
0 &\text{ if } t_i < x_i\\
t_i - x_i &\text{if } t_i \geq x_i
\end{cases} \\
& y_{ijk} \in \{0, 1\}\\
& \sum_{i = 1}^6 y_{ijk} = 1\\
& \sum_{i = 1}^6 x_i = 213.5\\
& x_i \geq 0\\
i &= 1, 2, ..., 6 \;\;\;\; j = 1, 2, ..., 5 \;\;\;\; k = 1, 2, ..., 5
\end{align}
$$


#### (2) Python code

##### Preparation & Parameter Definitions

In [1]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np

In [2]:
# Create list of timetable
timetable = [[9, 9, 9, 9, 7.5],
             [9, 9, 9, 9, 7.5],
             [9, 9, 9, 9, 7.5],
             [9, 9, 9, 9, 7.5],
             [9, 8, 8, 8, 6.5]]

In [3]:
# i departments
dept = 6
# j rooms
room = 5
# k days
day = 5

In [4]:
#Create target share list
share =  np.array([92, 8, 48, 14, 10, 18], dtype=float)
share /= 190

# Calculate target total hours for each department (t_i)
target = []
for i in range(dept):
    val = float(213.5 * share[i])
    target.append(val)
target

[103.37894736842105,
 8.989473684210527,
 53.93684210526316,
 15.731578947368419,
 11.236842105263158,
 20.226315789473684]

##### Create the model

In [5]:
# Define model
mod = Model()

# Define decision variable
s = mod.addVars(6, lb=0, vtype=GRB.CONTINUOUS)
x = mod.addVars(6, vtype=GRB.CONTINUOUS)
y = mod.addVars(dept, room, day, vtype = GRB.BINARY)


# Construct constraints
for i in range(dept):
    # Calculation of the total hours allocated to each department
    mod.addConstr(sum(y[i, j, k] * timetable[j][k] for j in range(room) for k in range(day)) == x[i])
    # Underallocation, s
    mod.addConstr(s[i] >= target[i] - x[i])
    # Non-negative constraint; ignore overallocation
    mod.addConstr(s[i] >= 0)

# Ensure each room on each day is allocated to one department only & total adds up to 213.5 hrs
for j in range(room):
    for k in range(5):
        mod.addConstr(sum(y[i, j, k] for i in range(dept)) == 1)
        
# Non-negative Constraint
for i in range(dept):
    mod.addConstr(x[i] >= 0)

    
# Objective function
mod.setObjective(sum((s[i]/target[i] for i in range(dept))), GRB.MINIMIZE)

mod.update()

mod.optimize()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-01-10
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.6.0 22G320)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 49 rows, 162 columns and 330 nonzeros
Model fingerprint: 0x1ad83627
Variable types: 12 continuous, 150 integer (150 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 18 rows and 6 columns
Presolve time: 0.00s
Presolved: 31 rows, 156 columns, 306 nonzeros
Variable types: 1 continuous, 155 integer (150 binary)
Found heuristic solution: objective 0.6786365
Found heuristic solution: objective 0.1584360

Root relaxation: objective 0.000000e+00, 58 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Wor

</br>

#### (3) Reporting - allocation hours & timetable

In [6]:
# Readable format 
if mod.status == GRB.OPTIMAL:
    print(f'Optimal objective value: {mod.objVal}\n')
    print(f'{"DEPT":<6}{"Allocated":<10}{"Underallocation":^20}')
    print()
    for i in range(dept):
        allocated = x[i].X
        underalloc = s[i].X
        under_percent = (underalloc / target[i]) * 100 if  s[i].X != 0 else 0
        print(f'{i+1:<6}{allocated:>9.1f}{underalloc:>9.1f}{under_percent:>7.1f}%')
        print()
else:
    print('No optimal solution found')

Optimal objective value: 0.052031361155899465

DEPT  Allocated   Underallocation   

1          98.0      5.4    5.2%

2           9.0      0.0    0.0%

3          54.0      0.0    0.0%

4          16.0      0.0    0.0%

5          14.0      0.0    0.0%

6          22.5      0.0    0.0%



In [7]:
room_allocations = mod.getAttr('x', y)

allocation_table = [[0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0]]

for i in range(dept):
    for j in range(room):
        for k in range(day):
            if room_allocations[i, j, k] > 0.9:
                allocation_table[j][k] = i+1

# Print Table in Readable Format
print('New Allocation Timetable for Departments (Dept. Indexed 1-6):')
print()
days = {0: 'Mon', 1:'Tue', 2: 'Wed', 3: 'Thu', 4: 'Fri'}               

print('         ', end='')
for k in range (day):
    print(f'{days[k]:>4}', end=' ')
print()

for j in range(room):
    print (f'Room {j+1}: ', end=' ')
    for k in range(day):
        print(f'{allocation_table[j][k]:>4}', end=' ')
    print()


New Allocation Timetable for Departments (Dept. Indexed 1-6):

          Mon  Tue  Wed  Thu  Fri 
Room 1:     1    1    1    1    6 
Room 2:     3    3    3    3    6 
Room 3:     3    3    1    1    6 
Room 4:     1    2    1    1    5 
Room 5:     1    1    4    4    5 


</br>

### Floor Constraints
Rooms 1 and 2 are on the first floor, 3 and 4 are on the second, and 5 is on the third floor.  
If no department can be split between two or more floors on the same day, how will the schedule change?

#### (1) Optimization model

$$
\begin{align}
s_i &= \text{underallocation of department } i \text{ (total number of hours below the target)} \\
t_i &= \text{target hours that department } i \text{ should receive} \\
x_i &= \text{the new no. of hours allocated to department } i \\
y_{ijk} &= 1 \text{ if department } i \text{ is allocated to room } j \text{ in day } k, 0 \text{ otherwise} \\
f_{ikl} &= 1 \text{ if department } i \text{ is allocated on floor } l \text{ on day k}, 0 \text{ otherwise}\\
\end{align}
$$


$$
\begin{align}
\underset{{\bf s}}{\text{min}} \;\; & \sum_{i = 1}^6 \dfrac{s_i}{t_i} \\
\text{s.t.} \;\; & s_i =\begin{cases} 
0 &\text{ if } t_i \le x_i\\
t_i - x_i &\text{if } t_i \geq x_i
\end{cases} \\
& y_{ijk} \in \{0, 1\}\\
& \sum_{i = 1}^6 y_{ijk} = 1\\
& x_i \geq 0\\
& f_{ikl} \in \{0, 1\}\\
& \sum_{l=1}^3 f_{ikl} \le 1\\
i &= 1, 2, ..., 6 \;\;\;\; j = 1, 2, ..., 5 \;\;\;\; k = 1, 2, ..., 5 \;\;\;\; l = 1, 2, 3
\end{align}
$$


#### (2) Python code

In [8]:
# Define variable for floor
floor = 3
f = mod.addVars(dept, day, floor, vtype = GRB.BINARY)

# room j to floor l mapping
room_floor = {0: 0,
              1: 0,
              2: 1,
              3: 1,
              4: 2}

# Floor Constraint
for i in range(dept):
    for k in range(day):
        #linking room assignment variable y to floor assignment variable f
        for l in range(floor):
            # on day k, department i can only be assigned to rooms j that are on a floor l
            mod.addConstr(sum(y[i, j, k] for j in range(room) if room_floor[j] == l) <= 2 * f[i, k, l])
        # one department assigned to the maximum of 1 floor each day
        mod.addConstr(sum(f[i, k, l] for l in range(floor)) <= 1)           


# Update and Optimize the new model
mod.update()
mod.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 22.6.0 22G320)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 169 rows, 252 columns and 660 nonzeros
Model fingerprint: 0x917468fa
Variable types: 12 continuous, 240 integer (240 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 78 rows and 66 columns
Presolve time: 0.00s
Presolved: 91 rows, 186 columns, 516 nonzeros
Variable types: 1 continuous, 185 integer (180 binary)
Found heuristic solution: objective 0.9843193
Found heuristic solution: objective 0.3890176

Root relaxation: objective 1.584360e-01, 103 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl 

</br>

#### (3) Reporting - allocation hours & timetable

In [9]:
if mod.status == GRB.OPTIMAL:
    print(f'Optimal objective value: {mod.objVal}\n')
    print(f'{"DEPT":<6}{"Allocated":<10}{"Underallocation":^20}')
    print()
    for i in range(6):
        allocated = x[i].X
        underalloc = s[i].X
        under_percent = (underalloc / target[i]) * 100 if  s[i].X != 0 else 0
        print(f'{i+1:<6}{allocated:>9.1f}{underalloc:>9.1f}{under_percent:>7.1f}%')
        print()
else:
    print('No optimal solution found')

Optimal objective value: 0.15843600448019549

DEPT  Allocated   Underallocation   

1          87.0     16.4   15.8%

2           9.0      0.0    0.0%

3          60.0      0.0    0.0%

4          18.0      0.0    0.0%

5          14.5      0.0    0.0%

6          25.0      0.0    0.0%



In [10]:
room_allocations2 = mod.getAttr('x', y)

allocation_table2 = [[0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0]]

for i in range(6):
    for j in range(5):
        for k in range(5):
            if room_allocations2[i, j, k] > 0.9:
                allocation_table2[j][k] = i+1

# Print Table in Readable Format
print('New Allocation Timetable for Departments with Floor Constraints (Dept. Indexed 1-6):')
print()
print('         ', end='')
for k in range (day):
    print(f'{days[k]:>4}', end=' ')
print()

for j in range(room):
    print (f'Room {j+1}: ', end=' ')
    for k in range(day):
        print(f'{allocation_table2[j][k]:>4}', end=' ')
    print()

New Allocation Timetable for Departments with Floor Constraints (Dept. Indexed 1-6):

          Mon  Tue  Wed  Thu  Fri 
Room 1:     4    1    6    1    3 
Room 2:     2    1    3    1    3 
Room 3:     1    3    1    3    1 
Room 4:     1    4    1    3    1 
Room 5:     3    6    5    6    5 
